## Visualize LiDAR Scattered Point Elevation Data

This notebook uses datashader to visualize LiDAR elevation data from [the Puget Sound LiDAR consortium](http://pugetsoundlidar.ess.washington.edu/), a source of LiDAR data for the Puget Sound region of Washington, U.S.

The data are downloaded in 7z archives, then unzipped and visualized below.

## Setup

The following cell only needs to be run once on your new datashader conda environment.  The commands will:

 * Run the `download_sample_data.py` script to download LiDAR from [Puget Sound LiDAR consortium](http://pugetsoundlidar.ess.washington.edu)
 * Install `pylzma` for 7zip archive reading [the LiDAR files are .7z 7zip compressed files.
 * Make sure you're runninng `jupyter notebook` from the `examples` directory of your `datashader` clone.

In [1]:
# ! cd datashader/examples && python download_sample_data.py
# ! pip install pylzma

*Grid_Coordinate_System_Name*: State Plane Coordinate System

*State_Plane_Coordinate_System*: SPCS_Zone_Identifier Washington North, FIPS 4601

*Lambert_Conformal_Conic*:
 * Standard_Parallel: 47.500000
 * Standard_Parallel: 48.733333
 * Longitude_of_Central_Meridian: -120.833333
 * Latitude_of_Projection_Origin: 47.000000
 * False_Easting: 1640416.666667
 * False_Northing: 0.000000
    
http://www.spatialreference.org/ref/esri/102348/

Washington State Plane North - FIPS 4601

In [2]:
import sys
import glob
import os
from bokeh.models import WMTSTileSource
from bokeh.models import WMTSTileSource
from cartopy import crs as ccrs
from dask.distributed import Client
from holoviews.operation import datashade
from pyproj import Proj, transform
import dask.dataframe as dd
import datashader as ds
import geoviews as gv
import geoviews as gv
import holoviews as hv
import numpy as np
import rasterio as rio
client = Client()

/Users/psteinberg/anaconda3/envs/earth-env-36/lib/python3.6/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The spectral and spectral_r colormap was deprecated in version 2.0. Use nipy_spectral and nipy_spectral_r instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [5]:
lidar_7z_files = glob.glob(os.path.join(lidar_7z_dir, '*7z'))
lidar_gnd_files = unzip_all(lidar_7z_files)[:4]
lidar_gnd_files

FAILED ON 7Z data/puget_sound_lidar-2/BE_ASCII/q47122e3114.7z


['data/puget_sound_lidar-2-gdb/BE_ASCII/q47122d2101.gnd',
 'data/puget_sound_lidar-2-gdb/BE_ASCII/q47122d2102.gnd',
 'data/puget_sound_lidar-2-gdb/BE_ASCII/q47122d2103.gnd',
 'data/puget_sound_lidar-2-gdb/BE_ASCII/q47122d2104.gnd']

In [6]:
df_raw = dd.read_csv(lidar_gnd_files)

In [7]:
df_raw.head()

,X,Y,Z
0,1291149.60,181033.64,467.95
1,1291113.29,181032.53,460.24
2,1291065.38,181035.74,451.41
3,1291113.16,181037.32,455.51
4,1291116.68,181037.42,456.20


In [8]:
FT_2_M = 0.3048
washington_state_plane = Proj(init='epsg:2855')
web_mercator = Proj(init='epsg:3857')
    
def convert_coords(ddf):
    lon, lat = transform(washington_state_plane, web_mercator,ddf.X.values * FT_2_M, ddf.Y.values * FT_2_M)
    ddf['meterswest'], ddf['metersnorth'] = lon, lat 
    ddf2 = ddf[['meterswest', 'metersnorth', 'Z']].copy()
    del ddf
    return ddf2

import dask
import pandas as pd
@dask.delayed
def read_gnd(fname):
    return convert_coords(pd.read_csv(fname))

In [9]:
df = dd.from_delayed([read_gnd(f) for f in lidar_gnd_files])
df.head()

,meterswest,metersnorth,Z
0,-1.360837e+07,6.022013e+06,467.95
1,-1.360838e+07,6.022013e+06,460.24
2,-1.360840e+07,6.022014e+06,451.41
3,-1.360838e+07,6.022015e+06,455.51
4,-1.360838e+07,6.022015e+06,456.20


In [10]:
hv.notebook_extension('bokeh', width=95)

%opts Overlay [width=800 height=455 xaxis=None yaxis=None show_grid=False] 
%opts Shape (fill_color=None line_width=1.5) [apply_ranges=False] 
%opts Points [apply_ranges=False] WMTS (alpha=0.5) NdOverlay [tools=['tap']]

From the metadata for this Puget Sound Lidar Consortium data set:

```
West_Bounding_Coordinate: -123.12157
East_Bounding_Coordinate: -121.78725
North_Bounding_Coordinate: 47.94147
South_Bounding_Coordinate: 47.35807
```

In [11]:
kdims = ['meterswest', 'metersnorth',]
min_x, min_y = web_mercator(-122.6, 47.4)   # From the data set metadata, but could be df.min() or .max() 
max_x, max_y = web_mercator(-122, 47.8) 
dataset = gv.Dataset(df, kdims=kdims, vdims=['Z'])
x_range, y_range = ((min_x, max_x), (min_y, max_y))
shade_defaults = dict(x_range=x_range, y_range=y_range, x_sampling=12, y_sampling=12, width=800, height=455)
tri = hv.Points(dataset, kdims=kdims, vdims=['Z'])
shaded = datashade(tri, **shade_defaults)

In [12]:
tiles = gv.WMTS(WMTSTileSource(url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'))

In [13]:
shaded * tiles

b':DynamicMap   []'